# Deliverable 1. Retrieve Weather Data.
---
1. Create a folder called `Weather_Database` to save all the files related with this deliverable.

2. Save the `Weather_Database_starter_code.ipynb` starter code to the `Weather_Database` folder and rename it as `Weather_Database.ipynb`.

3. Use the `np.random.uniform` function to generate a new set of 2,000 random latitudes and 2,000 longitudes.

4. Use the `citipy` module to get the nearest city for each latitude and longitude combination.

5. Import your OpenWeatherMap's API key and assemble the API call URL as a string variable. Recall to edit the `config.py` file to add your API key; also, it's critical to avoid publishing your API key on your GitHub repository.

6. Retrieve the following information from the API call:

    * Latitude and longitude

    * Maximum temperature

    * Percent humidity

    * Percent cloudiness

    * Wind speed

    * Weather description (for example, clouds, fog, light rain, clear sky)

7. Add the weather data to a new DataFrame.

8. Export the DataFrame as a CSV file, and save it as `WeatherPy_Database.csv` in the `Weather_Database` folder.
---

In [1]:
# Import initial libraries
import numpy as np
import pandas as pd
import timeit
import random
from citipy import citipy
# Import the API key.
import os
from dotenv import load_dotenv
load_dotenv()

google_API_Key=os.getenv("google_API_Key")

In [2]:
type(google_API_Key)

str

## Use the `np.random.uniform` function to generate a new set of 2,000 random latitudes and 2,000 longitudes.

In [3]:
# Create a set of random latitude and longitude combinations
lats = np.random.uniform(-90.000, 90.000, size=2000)
lngs = np.random.uniform(-1800.000, 180.000, size=2000)

# Use the zip function to create an iterator of tuples containing the latitude and longitude combinations
lat_lngs = zip(lats,lngs)
lat_lngs

In [4]:
# Add the latitudes and longitudes to a list
coordinates = list(lat_lngs)
print(coordinates)

[(-20.261920808164675, -1293.345561745576), (-75.6852900231388, -690.8671947352768), (-65.56016242864004, -1367.9125109481652), (65.45816180856065, -990.4163467387342), (88.50079359654791, -1525.8016140898555), (-71.28340183111179, -885.9197207185881), (-70.63357792793731, -908.5842622348272), (-85.97940615061783, -1313.3585341092105), (7.6507554087706495, -152.96466047521403), (-61.224233506778084, -1009.7255677746256), (-62.89359597800414, 29.133933887876765), (35.61223450272347, -1281.8848512287707), (66.0901760720293, -559.9656265396193), (-79.847353769838, -1422.5839847149398), (-80.79696311333228, 42.23418188253709), (-79.45818835135479, -394.56186910966494), (-75.89010063605053, -104.50931309289854), (-16.990749787171282, -1759.1935162857847), (38.01391967215915, -142.70351359544475), (9.924205262317415, -908.5242466875638), (-37.20120446674366, -1746.4452803266317), (-60.78840825136173, -156.7510840202233), (47.04878187980296, -946.1328986886969), (-62.64981470016239, 162.70888

## Use the `citipy` module to get the nearest city for each latitude and longitude combination.

In [5]:
# Create an empty list for holding the cities
cities = []

# Use a for loop to identify nearest city for each latitude and longitude combination using the citipy module
for coordinate in coordinates:
    # Use the citipy module to identify the nearest city to the current coordinate
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

197

## Import your OpenWeatherMap's API key and assemble the API call URL as a string variable. Recall to edit the `config.py` file to add your API key.

In [6]:
# Import the requests library
import requests

# Import the time library
import time

# Import the datetime module from the datetime library
from datetime import datetime

# Import the OpenWeatherMap's API key
# Import the API key.
import os
from dotenv import load_dotenv
load_dotenv()

weather_API_KEY=os.getenv("weather_API_KEY")

In [7]:
# Assemble the OpenWeatherMap's API call
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_API_KEY

## Retrieve the following information from the API call.

- The latitude and longitude
- The Max temperature
- The % humidity
- The % cloudiness
- The Wind speed
- The Weather description, i.e., cloudy, fog, light rain, clear sky, etc. 

In [8]:
# Create an empty list to hold weather data for each city
city_data = []

# Print a message to indicate that the data retrieval starts
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters and set them to 1
record_count = 1
set_count = 1

# Loop through all the cities in our list to fetch weather data for each city
for i, city in enumerate(cities):
        
    # Group cities in sets of 50 for logging purposes
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create an endpoint URL for each city
    city_url = url + "&q=" + city.replace(" ","+")
    
    # Log the url, record, and set numbers
    print(f"Processing Record {record_count} of Set {set_count} | {city}")

    # Add 1 to the record count
    record_count += 1

    # Run an API request for each of the cities
    try:
        city_weather = requests.get(city_url).json()
        # Parse out the latitude, longitude, max temp, humidity, cloudiness, wind, country, and weather description
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_description=city_weather["weather"][0]["description"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')    
        # Append the city information into the city_data list
        city_data.append({"City": city.title(),
                          "Cloudiness": city_clouds,
                          "Country": city_country,
                          "Current Description": city_description,
                          "Humidity": city_humidity,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Wind Speed": city_wind})
    
    # If an error is experienced, skip the city
    except:
        print("City not found. Skipping...")
        pass

# Indicate that the data retrieval is complete 
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | halalo
City not found. Skipping...
Processing Record 2 of Set 1 | mys shmidta
City not found. Skipping...
Processing Record 3 of Set 1 | hilo
Processing Record 4 of Set 1 | port elizabeth
Processing Record 5 of Set 1 | port alfred
Processing Record 6 of Set 1 | vaini
Processing Record 7 of Set 1 | punta arenas
Processing Record 8 of Set 1 | fortuna
Processing Record 9 of Set 1 | mataura
Processing Record 10 of Set 1 | bluff
Processing Record 11 of Set 1 | taolanaro
City not found. Skipping...
Processing Record 12 of Set 1 | egvekinot
Processing Record 13 of Set 1 | kosa
Processing Record 14 of Set 1 | albany
Processing Record 15 of Set 1 | vaitupu
City not found. Skipping...
Processing Record 16 of Set 1 | qaanaaq
Processing Record 17 of Set 1 | mount isa
Processing Record 18 of Set 1 | rikitea
Processing Record 19 of Set 1 | castro
Processing Record 20 of Set 1 | ushuaia
Processing Record 21 of 

Processing Record 39 of Set 4 | husavik
Processing Record 40 of Set 4 | dandong
Processing Record 41 of Set 4 | port-de-paix
City not found. Skipping...
Processing Record 42 of Set 4 | fort walton beach
Processing Record 43 of Set 4 | sitka
Processing Record 44 of Set 4 | australind
Processing Record 45 of Set 4 | kloulklubed
Processing Record 46 of Set 4 | sentyabrskiy
City not found. Skipping...
Processing Record 47 of Set 4 | mount gambier
-----------------------------
Data Retrieval Complete      
-----------------------------


In [9]:
# Print the length of the city_data list to verify how many cities you have
len(city_data)

180

## Add the weather data to a new DataFrame.

In [10]:
# Use the city_data list to create a new pandas DataFrame.
city_data_df = pd.DataFrame(city_data)

In [11]:
# Display sample data
city_data_df.head(5)

,City,Cloudiness,Country,Current Description,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Hilo,0,US,clear sky,72,19.7297,-155.0900,80.33,6.91
1,Port Elizabeth,0,ZA,clear sky,77,-33.9180,25.5701,61.11,9.22
2,Port Alfred,30,ZA,scattered clouds,86,-33.5906,26.8910,60.19,3.00
3,Vaini,20,TO,few clouds,74,-21.2000,-175.2000,82.56,5.75
4,Punta Arenas,40,CL,scattered clouds,66,-53.1500,-70.9167,46.51,23.02


In [12]:
# Display the DataFrame's column names using the columns Pandas function
city_data_df.columns

Index(['City', 'Cloudiness', 'Country', 'Current Description', 'Humidity',
       'Lat', 'Lng', 'Max Temp', 'Wind Speed'],
      dtype='object')

In [13]:
# Create a list to reorder the column names as follows:
# "City", "Country", "Lat", "Lng", "Max Temp", "Humidity",  "Cloudiness", "Wind Speed",  "Current Description"
# Recreate the DataFrame by using the new column order
# Display sample data
new_column_order = ["City", "Country","Lat","Lng","Max Temp","Humidity","Cloudiness","Wind Speed","Current Description"]
city_data_df= city_data_df[new_column_order]
city_data_df.head(10)      

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Hilo,US,19.7297,-155.0900,80.33,72,0,6.91,clear sky
1,Port Elizabeth,ZA,-33.9180,25.5701,61.11,77,0,9.22,clear sky
2,Port Alfred,ZA,-33.5906,26.8910,60.19,86,30,3.00,scattered clouds
3,Vaini,TO,-21.2000,-175.2000,82.56,74,20,5.75,few clouds
4,Punta Arenas,CL,-53.1500,-70.9167,46.51,66,40,23.02,scattered clouds
5,Fortuna,US,40.5982,-124.1573,52.14,81,0,4.61,clear sky
6,Mataura,NZ,-46.1927,168.8643,71.82,49,100,1.41,light rain
7,Bluff,NZ,-46.6000,168.3333,58.33,76,100,13.69,overcast clouds
8,Egvekinot,RU,66.3167,-179.1667,27.95,99,100,20.45,snow
9,Kosa,RU,59.9445,54.9977,3.27,90,67,4.41,broken clouds


In [14]:
# Display the data types of each column by using the dtypes Pandas function
city_data_df.dtypes

City                    object
Country                 object
Lat                    float64
Lng                    float64
Max Temp               float64
Humidity                 int64
Cloudiness               int64
Wind Speed             float64
Current Description     object
dtype: object

## Export the DataFrame as a CSV file, and save it as `WeatherPy_Database.csv` in the `Weather_Database` folder.

In [24]:
# Set the output file name
output_data_file = "/Users/ericagutierrez/Desktop/AnalysisProjects/World_Weather_Analysis/World_Weather_Analysis/Weather_Database/WeatherPy_Database.csv"
# Export the city_data DataFrame into a CSV file
city_data_df.to_csv(output_data_file, index_label="City_ID")